<a href="https://colab.research.google.com/github/AAnirudh07/Aposemat-IoT23-Network-Classification/blob/main/IOT23_Compilation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#File Read
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np
import csv
import os
 
main="/content/drive/MyDrive/IoTScenarios"
fols=[main+"/"+i for i in os.listdir(main)]
files=[]
for i in fols:
    if("points" in i or "csv" in i):
      continue
    F=os.path.join(i,os.listdir(i)[0])
    while(F.find("conn.log.labeled")==-1):
        F=F+"//"+os.listdir(F)[0]
    files.append(F)
 
label='tunnel_parents   label   detailed-label'
j=0
Dict=dict()
li=[]
li2=[]
for fi in files:
    j=0
    tsv_file = open(fi)
    read_tsv = csv.reader(tsv_file, delimiter="\t")
    for i in read_tsv:
        j=j+1
        if(j==7):
            li=i
        if(j==8):
            li2=i
            for key in range(1,len(li)):
                Dict[li[key]]=li2[key]
        if(j==9):
            break
#open(os.path.join(main,'my_csv.csv'),"w").close()
#CONCATTING CHANGE
 
frames=[]
j=0
print("All cols:",Dict.keys())
cols=['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label']
cols12 = ["ts","uid","id.orig_h","id.orig_p","id.resp_h","id.resp_p","proto","service","duration","orig_bytes","resp_bytes","conn_state","local_orig","local_resp","missed_bytes","history","orig_pkts","orig_ip_bytes","resp_pkts","resp_ip_bytes","label"]
print(cols)
for fi in files:
    df = pd.read_csv(fi, sep="\t",skiprows=10,names=cols12,nrows=10000)
    df.rename(columns={label:'label'},inplace=True)
    df.drop(df.tail(1).index,inplace=True)
    df.label = df.label.apply(lambda x: 'B' if ('benign' in x or 'Benign' in x) else 'A')
    #df.to_csv(os.path.join(main,'my_csv.csv'), mode='a', header=False)
    frames.append(df)
  
#CONCATTING CHANGE
df=pd.concat(frames)
print(len(df))# nrows takes min(nrows,actual)
# df = pd.get_dummies(df, columns=['proto'])
# df = pd.get_dummies(df, columns=['conn_state'])
# df = pd.get_dummies(df, columns=['service'])


main_train=['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']
#actual_col=Dict.keys()
actual_col=df.columns
X_cols=['label']
for i in main_train:
    if(i in actual_col):
        X_cols.append(i)
 
print("X_cols",X_cols)
df.drop_duplicates(keep="first",inplace=True)
 
X_train=df[X_cols]
 
 
drop_cols=[]
#Directly read from 
output="label" 
if(False):
  X_train=pd.read_csv("/content/drive/MyDrive/IOT23/iot_traindata.csv")
#comment to include from main folder
 
 
 
########################################################################
# finding mean of duration
temp_duration  = X_train.loc[X_train['duration']!='-','duration'].astype('float16')
index_inf_duration = temp_duration[temp_duration.isin([-np.inf, np.inf])]
#print("Hello",index_inf_duration)
temp_duration.drop([448,449], inplace = True)
avg_duration = np.asarray(temp_duration).mean()
X_train.loc[(X_train['duration']=='-'), 'duration'] = str(avg_duration )
# drop missed bytes column
X_train.drop([448,449], inplace = True)
#X_train.drop(columns=['missed_bytes'], inplace=True)
X_train['duration'] = X_train['duration'].astype('float16')
 
# detecting outliers
# #   w.r.t duration
# Q1 = np.percentile(X_train['duration'], 25, interpolation = 'midpoint')   
# Q3 = np.percentile(X_train['duration'], 75, interpolation = 'midpoint') 
# IQR = Q3 - Q1 
# upper = np.where(X_train['duration'] >= (Q3+1.5*IQR))
# lower = np.where(X_train['duration'] <= (Q1-1.5*IQR))
# print(type(upper[0]))
# X_train.drop(upper[0][1], inplace=True)
# X_train.drop(lower, inplace=True)
X_train.reset_index(drop=True)
# Change in #rows after removing outliers : 31613 -> 28700
#########################################################################
X_train=X_train.replace(to_replace ="-",value ="0")
 
#output="output"
Y = X_train[[output]]
Y_train=Y
X_train.drop(columns=[output], inplace=True)
X_train['duration'] = X_train['duration'].astype('float64')
X_train['orig_bytes'] = X_train['orig_bytes'].astype('float64')
X_train['resp_bytes'] = X_train['resp_bytes'].astype('float64')
X=X_train

All cols: dict_keys(['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'local_orig', 'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'tunnel_parents   label   detailed-label'])
['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label']
173084
X_cols ['label', 'duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes']


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [ ]:
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
os.chdir('/content/drive/My Drive/IoTScenarios')
print("Change successful.")
import glob
file_names = [i for i in glob.glob('**/**/*')]
file_names[-3]+='/conn.log.labeled'
#file_names+='/conn.log.labeled'
file_names
#now combine all the files into 1 single dataframe
cols = ["ts","uid","id.orig_h","id.orig_p","id.resp_h","id.resp_p","proto","service","duration","orig_bytes","resp_bytes","conn_state","local_orig","local_resp","missed_bytes","history","orig_pkts","orig_ip_bytes","resp_pkts","resp_ip_bytes","label"]
iotdf = pd.DataFrame(columns=cols)
# for file in file_names:

for file in file_names: 
  currdf = pd.read_csv(file,sep='\t',nrows=10000,names=cols,skiprows=10)
  currdf.drop(currdf.tail(1).index,inplace=True)
  iotdf = pd.concat([iotdf,currdf])
print("Concatenation complete.")
iotdf['service'].replace('-','new_service',inplace=True)
niotdf = iotdf.drop(columns=['ts','uid','local_orig','local_resp','missed_bytes'])

import numpy as np
niotdf.replace('-', np.nan,inplace=True)
ans = niotdf.groupby('label')
clean_niotdf = niotdf.apply(lambda x: x.fillna(x.value_counts().index[0]))
iotcp = clean_niotdf.copy()
iotcp = iotcp.replace(to_replace ='.*[bB]enign.*', value = 'Benign', regex = True)
iotcp = iotcp.replace(to_replace ='.*Malicious.*', value = 'Malicious', regex = True)
iotcp['duration'] = iotcp['duration'].astype('float64')
iotcp['orig_bytes'] = iotcp['orig_bytes'].astype('float64')
iotcp['resp_bytes'] = iotcp['resp_bytes'].astype('float64')
#import the necessary libraries
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
import numpy as np
np.random.seed(0)
#train_iot.shape
print(iotcp.columns)
iotcp = iotcp.drop(columns=['id.orig_h','id.resp_h'])
iotcp= pd.get_dummies(iotcp, columns=['proto'])
iotcp = pd.get_dummies(iotcp, columns=['conn_state'])
iotcp = pd.get_dummies(iotcp, columns=['service'])
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
y = iotcp['label']
x = iotcp.drop(columns=['label'])
print(x.head())
Y=y
X=x

Mounted at /content/drive
Change successful.
Concatenation complete.
Index(['id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto', 'service',
       'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'history',
       'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label'],
      dtype='object')
   id.orig_p  id.resp_p  ...  service_ssh  service_ssl
0    41101.0       23.0  ...            0            0
1    60905.0       23.0  ...            0            0
2    44301.0       23.0  ...            0            0
3    50244.0       23.0  ...            0            0
4    34243.0    49560.0  ...            0            0

[5 rows x 33 columns]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

import numpy as np

import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score

from sklearn.tree import DecisionTreeClassifier

from scipy import stats

import time

from sklearn.preprocessing import RobustScaler

In [ ]:
#Decision Tree

#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, shuffle=True ,test_size=0.2)
DT = DecisionTreeClassifier()
start = time.time()
print('program start...')
print()
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
np.random.seed(0)
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size =0.3,stratify=Y,random_state=10)
ydf = pd.DataFrame(ytrain)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in xtrain.columns:
        if xtrain[column_name].dtype == object:
            print(column_name)
            xtrain[column_name] = le.fit_transform(xtrain[column_name])

        else:
            pass
for column_name in xtest.columns:
        if xtest[column_name].dtype == object:
            print(column_name)
            xtest[column_name] = le.fit_transform(xtest[column_name])

        else:
            pass

ydf['label'] = le.fit_transform(ydf['label'])
ytest=le.fit_transform(ytest)
ytrain=ydf

Y_test=ytest
X_test=xtest
X_train=xtrain
Y_train=ytrain

DT.fit(X_train, Y_train)
print()

print('prediction:')
y_pred = DT.predict(X_test)
train_pred=DT.predict(X_train)
print(train_pred[train_pred=='1'])
print(len(y_pred))
print("size of Y_test",len(Y_test))


print('Score:')
score = DT.score(X_test,Y_test)
print(score)

# y_pred = y_pred.astype('int16')
# Y_test = Y_test.astype('int16')


end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

print("----Accuracy-----  ")
print(accuracy_score( Y_test , y_pred ))
print('----F1 score-----  ')
print(f1_score( Y_test , y_pred ))
print('----Recall------- ')
print(recall_score(Y_test , y_pred ))
print('----Precision----')
print(precision_score( Y_test , y_pred))

program start...

history
history


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



prediction:
[]
51926
size of Y_test 51926
Score:
0.9879251242152294
program end...

time cost: 
0.746208667755127 seconds
----Accuracy-----  
0.9879251242152294
----F1 score-----  
0.992368456285982
----Recall------- 
0.9953365725028689
----Precision----
0.9894179894179894


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [ ]:
ydf = pd.DataFrame(ytrain)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in xtrain.columns:
        if xtrain[column_name].dtype == object:
            print(column_name)
            xtrain[column_name] = le.fit_transform(xtrain[column_name])

        else:
            pass

ydf['label'] = le.fit_transform(ydf['label'])
ytrain=ydf

In [ ]:
path = DT.cost_complexity_pruning_path(X_train, Y_train)
path


{'ccp_alphas': array([0.        , 0.32387924]),
 'impurities': array([0.        , 0.32387924])}

In [ ]:
#SVM
import sklearn.metrics as skm
print(df.shape)
print(X_cols)
#SVM_classifier
drop_cols=[]
SVM_classifier = SVC()
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
np.random.seed(0)
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size =0.3,stratify=Y,random_state=10)
ydf = pd.DataFrame(ytrain)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in xtrain.columns:
        if xtrain[column_name].dtype == object:
            print(column_name)
            xtrain[column_name] = le.fit_transform(xtrain[column_name])

        else:
            pass
for column_name in xtest.columns:
        if xtest[column_name].dtype == object:
            print(column_name)
            xtest[column_name] = le.fit_transform(xtest[column_name])

        else:
            pass

ydf['label'] = le.fit_transform(ydf['label'])
ytest=le.fit_transform(ytest)
ytrain=ydf

Y_test=ytest
X_test=xtest
X_train=xtrain
Y_train=ytrain
start = time.time()
print('program start...')
print()

SVM_classifier = SVC(C=1.0, cache_size=700, verbose=True).fit(X_train, Y_train.values.ravel())
print()
print(SVM_classifier.score(X_test, Y_test))

y_pred = SVM_classifier.predict(X_test)
wee=skm.accuracy_score(Y_test,y_pred,normalize=False)
print("Acuracy",wee)
end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

print("Classifiction Report :")
print(classification_report(Y_test, y_pred))
print("----Accuracy-----  ")
print(accuracy_score( Y_test , y_pred ))
print('----F1 score-----  ')
print(f1_score( Y_test , y_pred))
print('----Recall------- ')
print(recall_score(Y_test , y_pred))
print('----Precision----')
print(precision_score( Y_test , y_pred))


(173084, 21)
['label', 'duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes']
history
history
program start...

[LibSVM]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0.7887570773793475
Acuracy 40957
program end...

time cost: 
2653.947418689728 seconds
Classifiction Report :
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     10969
           1       0.79      1.00      0.88     40957

    accuracy                           0.79     51926
   macro avg       0.39      0.50      0.44     51926
weighted avg       0.62      0.79      0.70     51926

----Accuracy-----  
0.7887570773793475
----F1 score-----  
0.8819051925540734
----Recall------- 
1.0
----Precision----
0.7887570773793475


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Rain Forest
#Rain forest is here
#import the necessary libraries
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
np.random.seed(0)
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size =0.3,stratify=Y,random_state=10)
clf = RandomForestClassifier(n_jobs=2,random_state=0)
ydf = pd.DataFrame(ytrain)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in xtrain.columns:
        if xtrain[column_name].dtype == object:
            #print(column_name)
            xtrain[column_name] = le.fit_transform(xtrain[column_name])

        else:
            pass
for column_name in xtest.columns:
        if xtest[column_name].dtype == object:
          xtest[column_name] = le.fit_transform(xtest[column_name])

        else:
            pass

ydf['label'] = le.fit_transform(ydf['label'])
ytest=le.fit_transform(ytest)
ytrain=ydf
clf.fit(xtrain,ytrain)
print(xtrain.shape)
predic_res = clf.predict(xtest)
ytestdf=ytest
print("----Accuracy-----  ")
print(accuracy_score( ytestdf , predic_res ))
print('----F1 score-----  ')
print(f1_score( ytestdf , predic_res))
print('----Recall------- ')
print(recall_score(ytestdf , predic_res))
print('----Precision----')
print(precision_score( ytestdf , predic_res))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


(121158, 33)
----Accuracy-----  
0.9882910295420406
----F1 score-----  
0.9925672371638141
----Recall------- 
0.9911858778719145
----Precision----
0.9939524520725705


In [ ]:
#1D convnet

#Cnn
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Y=pd.DataFrame(Y)
Y=le.fit_transform(Y)
Y=np.asarray(Y).astype(np.int32)
# Y=pd.DataFrame(Y)
X=pd.DataFrame(X)

#Y.astype('int32').dtypes
# print("weee",Y.dtypes,"hello", Y.shape)

X.history=le.fit_transform(X.history)
# print(Y.dtypes)
# print(X.history.head())
# print(X.head())
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,stratify=Y, test_size=0.2)

# cnn model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1],1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
import time
import tensorflow as tf

start = time.time()
tf.config.experimental_run_functions_eagerly(True)
print('program start...')
print()
siz_train=[X_train.shape[0],X_train.shape[1],1]
siz_test=[X_test.shape[0],X_test.shape[1],1]
for col in X_train:
  X_train[col] = np.asarray(X_train[col]).astype('float32')
  column = X_train[col]
  max_value = column.max()
  if(max_value>0):
    X_train[col] = X_train[col].apply(lambda x: x/max_value)
for col in X_test:
  X_test[col] = np.asarray(X_test[col]).astype('float32')
  column = X_test[col]
  max_value = column.max()
  if(max_value>0):
    X_test[col] = X_test[col].apply(lambda x: x/max_value)




x_train, x_test = X_train.values.reshape(siz_train), X_test.values.reshape(siz_test)
history = model.fit(x_train, Y_train, epochs = 20, batch_size=256, validation_data=(x_test,Y_test),verbose=1)
end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')





/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


program start...



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Epoch 1/20
  2/541 [..............................] - ETA: 28s - loss: 0.6360 - accuracy: 0.8184

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


541/541 [==============================] - 26s 49ms/step - loss: 0.2588 - accuracy: 0.9101 - val_loss: 0.2366 - val_accuracy: 0.9170
Epoch 2/20
541/541 [==============================] - 26s 48ms/step - loss: 0.2433 - accuracy: 0.9138 - val_loss: 0.2326 - val_accuracy: 0.9175
Epoch 3/20
541/541 [==============================] - 27s 50ms/step - loss: 0.2399 - accuracy: 0.9142 - val_loss: 0.2294 - val_accuracy: 0.9174
Epoch 4/20
541/541 [==============================] - 27s 50ms/step - loss: 0.2343 - accuracy: 0.9142 - val_loss: 0.2320 - val_accuracy: 0.9172
Epoch 5/20
541/541 [==============================] - 26s 49ms/step - loss: 0.2280 - accuracy: 0.9143 - val_loss: 0.2183 - val_accuracy: 0.9175
Epoch 6/20
541/541 [==============================] - 27s 49ms/step - loss: 0.2237 - accuracy: 0.9144 - val_loss: 0.2164 - val_accuracy: 0.9180
Epoch 7/20
541/541 [==============================] - 26s 49ms/step - loss: 0.2192 - accuracy: 0.9146 - val_loss: 0.2098 - val_accuracy: 0.9178
Epo

In [ ]:
model.save('/content/drive/MyDrive/IOT231dcnn')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/IOT231dcnn/assets


In [ ]:
print(history.history['accuracy'][-1],history.history['accuracy'][-2])

0.9375374913215637 0.9373857975006104


In [ ]:
while(history.history['accuracy'][-1]-history.history['accuracy'][-2]>0.001):
  history=model.fit(x_train, Y_train, epochs = 20, batch_size=256, validation_data=(x_test,Y_test),verbose=1)

Epoch 1/20
  2/541 [..............................] - ETA: 27s - loss: 0.1826 - accuracy: 0.9355

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


541/541 [==============================] - 27s 50ms/step - loss: 0.1838 - accuracy: 0.9344 - val_loss: 0.1973 - val_accuracy: 0.9358
Epoch 2/20
541/541 [==============================] - 27s 50ms/step - loss: 0.1834 - accuracy: 0.9346 - val_loss: 0.1791 - val_accuracy: 0.9380
Epoch 3/20
541/541 [==============================] - 26s 49ms/step - loss: 0.1812 - accuracy: 0.9349 - val_loss: 0.1789 - val_accuracy: 0.9378
Epoch 4/20
541/541 [==============================] - 27s 50ms/step - loss: 0.1814 - accuracy: 0.9349 - val_loss: 0.1768 - val_accuracy: 0.9385
Epoch 5/20
541/541 [==============================] - 27s 49ms/step - loss: 0.1811 - accuracy: 0.9349 - val_loss: 0.1781 - val_accuracy: 0.9376
Epoch 6/20
541/541 [==============================] - 26s 48ms/step - loss: 0.1799 - accuracy: 0.9351 - val_loss: 0.1850 - val_accuracy: 0.9375
Epoch 7/20
541/541 [==============================] - 26s 49ms/step - loss: 0.1794 - accuracy: 0.9352 - val_loss: 0.1772 - val_accuracy: 0.9378
Epo

KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
# predict probabilities for test set
#yhat_probs = model.predict(X_test)
# predict crisp classes for test set
X_test=x_test
yhat_classes = model.predict(X_test)
yhat_classes = np.where(yhat_classes > 0.5, 1, 0)
# reduce to 1d array
#yhat_probs = yhat_probs[:, 0]
print(yhat_classes)
yhat_classes = yhat_classes
print(len(yhat_classes))
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Y_test, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(Y_test, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(Y_test, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Y_test, yhat_classes)
print('F1 score: %f' % f1)
#https://machinelearningmastery.com/how-to-calculate-precision-recall-f1-and-more-for-deep-learning-models/
#https://stats.stackexchange.com/questions/258166/good-accuracy-despite-high-loss-value
#https://stats.stackexchange.com/questions/272754/how-do-you-interpret-the-cross-entropy-value

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


[[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]
34617
Accuracy: 0.939018
Precision: 0.930164
Recall: 0.997583
F1 score: 0.962695
